#Procesamiento del Lenguaje Natural -TUIA
---
## Trabajo Práctico  
**Autor**: López Crespo, Facundo

**Legajo**: L-3339/1


# Instalación de librerías

In [1]:
%%capture
! pip install selenium
! pip install langdetect
! pip install import os
import gdown
! pip install tensorflow tensorflow-text
! pip install pydub
! pip install SpeechRecognition
! pip install PyPDF2
! pip install chromadb
! pip install python-decouple
! pip install deep-translator
!pip install rdflib
!python -m spacy download es_core_news_sm
!pip install llama-index-llms-ollama
!pip install wikipedia
!pip install litellm[proxy]
!pip install llama-index
!pip install rank-bm25
!pip install nltk
!apt-get update
!pip install selenium webdriver-manager
!pip install beautifulsoup
!pip install youtube_transcript_api

# Importación de librerías

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import re
from langdetect import detect
import networkx as nx
import plotly.graph_objects as go
import numpy as np
import rdflib
import urllib.parse
from rdflib import Namespace
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text
import sentencepiece
from pydub import AudioSegment
import speech_recognition as sr
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
import PyPDF2
import chromadb
from chromadb.config import Settings
import requests
import json
from decouple import config, Config, Csv
from jinja2 import Template
import pandas as pd
from deep_translator import GoogleTranslator
from itertools import chain
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from huggingface_hub import InferenceClient
from typing import List, Dict, Any
import spacy
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.core.agent import ReActAgent
from llama_index.core.tools import FunctionTool
import wikipedia
from chromadb.config import Settings
from llama_index.core.agent.react.formatter import ReActChatFormatter
from rank_bm25 import BM25Okapi
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
from nltk.tokenize import word_tokenize
nltk.data.path.append('/root/nltk_data/tokenizers/punkt')
import ast
import pickle
from typing import List, Dict, Any
import os
import gdown
from google.colab import userdata
from youtube_transcript_api import YouTubeTranscriptApi
from sentence_transformers import SentenceTransformer, CrossEncoder


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [5]:
from google.colab import drive
drive.mount('/content/drive')
drive_path = '/content/drive/My Drive/tecnicatura_ia/procesamiento_lenguaje_natural/NLP-TUIA-LopezCrespo/'

Mounted at /content/drive


# Ejercicio 1 - RAG

## Modelo de embedding

In [6]:
# Cargar Universal Sentence Encoder
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")

## Parte 1: Base vectorial

La base de datos vectorial tendrá el fin de almacenar:

- Reglas del juego generales.
- Información sobre el juego en general. Tácticas que se pueden utilizar, diferectes campos de batalla, etc.
- Opiniones.


### Funciones

In [7]:
def extract_text_with_pypdf2(pdf_path):
  '''
  Descripción:
  La función `extract_text_with_pypdf2` extrae texto de un archivo PDF utilizando la biblioteca PyPDF2, limpia el texto extraído
  con la función `limpiar_texto` y concatena el contenido de todas las páginas en una sola cadena.

  -------------------------------------------------------------------------------------------------

  Parámetros:
  - pdf_path (str): Ruta del archivo PDF del cual se extraerá el texto.

  -------------------------------------------------------------------------------------------------

  Retorna:
  - str: Cadena que contiene el texto extraído y procesado de todas las páginas del PDF.

  -------------------------------------------------------------------------------------------------

  Procedimiento:
  1. Abre el archivo PDF en modo binario de solo lectura.
  2. Crea un objeto lector (`PdfReader`) para acceder al contenido del PDF.
  3. Itera sobre todas las páginas del PDF:
    - Extrae el texto de la página actual usando `extract_text()`.
    - Procesa el texto extraído con la función `limpiar_texto` para eliminar caracteres no deseados y formatearlo.
    - Sustituye saltos de línea (`\n`) por espacios y concatena el texto procesado a la variable `full_text`.
  4. Cierra el archivo PDF automáticamente al salir del bloque `with`.
  5. Devuelve la cadena completa con el texto extraído y procesado.
  '''

  full_text = ""
  with open(pdf_path, "rb") as pdf_file:
      reader = PyPDF2.PdfReader(pdf_file)
      for page in reader.pages:
          text = page.extract_text()
          text = limpiar_texto(text)
          if text:
              full_text +=  text.replace("\n", " ")
  return full_text


In [8]:
def limpiar_texto(texto):
    '''
    Descripción:
    La función `limpiar_texto` procesa una cadena de texto para eliminar caracteres no deseados, reducir espacios múltiples
    y conservar solo caracteres alfanuméricos, algunos signos de puntuación comunes y letras con acentos o diéresis.

    -------------------------------------------------------------------------------------------------

    Parámetros:
    - texto (str): Cadena de texto que se desea limpiar.

    -------------------------------------------------------------------------------------------------

    Retorna:
    - str: Cadena de texto procesada, sin caracteres no deseados, con un formato más uniforme y sin espacios redundantes.

    -------------------------------------------------------------------------------------------------

    Procedimiento:
    1. Elimina caracteres no codificados en formato Unicode (`uXXXX`) o hexadecimal (`xXX`) utilizando expresiones regulares.
    2. Conserva únicamente caracteres alfanuméricos, letras con acentos, diéresis y signos básicos de puntuación
      (espacios, comas, puntos, punto y coma, signos de exclamación/interrogación, guiones y paréntesis).
    3. Sustituye múltiples espacios consecutivos por un solo espacio.
    4. Quita los espacios al inicio y al final del texto.
    5. Devuelve la cadena procesada.
    '''
    # Eliminar caracteres no codificados (\uXXXX, \xXX, etc.)
    texto = re.sub(r'\\u[0-9a-fA-F]{4}|\\x[0-9a-fA-F]{2}', '', texto)
    # Eliminar espacios múltiples
    texto = re.sub(r'\s+', ' ', texto)
    # Quitar espacios iniciales y finales
    texto = texto.strip()
    return texto


In [9]:
def extraer_comentarios_foro(url):
    """
    Extrae los comentarios de un foro de BoardGameGeek desde la página dinámica usando Selenium.
    Utiliza un chromedriver local.

    Args:
        url (str): URL del hilo de foro.

    Returns:
        list: Lista de comentarios extraídos.
    """
    # Configurar las opciones del navegador
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--disable-gpu')

    # Configurar la ruta del driver
    service = Service('/usr/bin/chromedriver')

    driver = webdriver.Chrome(options=chrome_options)
    try:
        # Abrir la página web
        driver.get(url)

        # Esperar hasta que los comentarios estén visibles en la página
        WebDriverWait(driver, 20).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "gg-markup-safe-html"))
        )

        # Buscar todos los comentarios dentro de las etiquetas <gg-markup-safe-html>
        comentarios = driver.find_elements(By.CSS_SELECTOR, "gg-markup-safe-html")
        # Extraer el texto de cada comentario
        comentarios_texto = [comentario.text for comentario in comentarios]

        return comentarios_texto

    finally:
        # Cerrar el navegador
        driver.quit()

In [10]:
def extraer_reseña_blog(url):
    """
    Extrae la reseña volcada en el blog utilizando Beautiful Soup y delimitándose
    los párrafos que la contienen.
    Args:
        url (str): URL del blog donde se comenta el juego.

    Returns:
        str: texto de la reseña.
    """

    response = requests.get(url, verify=False)
    soup = BeautifulSoup(response.text, 'html.parser')
    # La reseña se encuentra debajo de un título h2.
    elementos_h2 = soup.find_all('h2')
    h2_reseña = None
    for elemento_h2 in elementos_h2:
      if elemento_h2.text =='Opinión Personal':
        h2_reseña = elemento_h2
    elementos_debajo_h2_reseña = h2_reseña.find_all_next()
    parrafos_textos_debajo_titulo_reseña = [elemento.text for elemento in elementos_debajo_h2_reseña if elemento.name =='p']
    # La reseña se encuentra encima de un título h3.
    elementos_h3 = soup.find_all('h3')
    h3_siguiente_reseña = None
    for elemento_h3 in elementos_h3:
      if elemento_h3.text =='Me gusta esto:':
        h3_siguiente_reseña = elemento_h3
    elementos_encima_h3_siguiente_reseña = h3_siguiente_reseña.find_all_previous()
    parrafos_textos_encima_titulo_siguiente_reseña = [elemento.text for elemento in elementos_encima_h3_siguiente_reseña if elemento.name =='p']
    # unión entre los dos conjuntos de párafos
    texto_reseña = ''
    for parrafo_texto in parrafos_textos_debajo_titulo_reseña:
      if parrafo_texto in parrafos_textos_encima_titulo_siguiente_reseña:
        texto_reseña += parrafo_texto
    return texto_reseña


In [11]:
def chuncker(text:str)->list[str]:
    '''
    Descripción:
    Divide un texto largo en fragmentos más pequeños, manteniendo una
    superposición entre ellos, utilizando un divisor de texto basado en el
    número de caracteres.
    '''

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=30)
    texts = text_splitter.split_text(text)
    return texts

In [12]:
def insertar_db(pedazos_documento:list, metadata:dict, base_id:int)->None:
    '''
    Inserta cada pedazo de un documento en la base de dato vectorial junto a
    la descripción(única para el documento), el identificador y el vector
    que corresponda. Si el trozo de texto no está en inglés lo traduce a este
    idioma
    '''

    i = 0
    for texto in pedazos_documento:
      current_id = f'{base_id}_{i}'  # Asegúrate de IDs únicos

      if detect(texto) != 'en':
            texto = GoogleTranslator(source=detect(texto), target='en').translate(texto)
      embedding = embed(texto).numpy().flatten().tolist() # Genera el embedding del documento

      # Depuración
      print(f"ID: {current_id}, Embedding Length: {len(embedding)}, Metadata: {metadata}, texto: {texto}")
      try:
          vec_db.add(
              documents=[texto],
              metadatas=[metadata],
              ids=[current_id],
              embeddings=[embedding]
          )
      except Exception as e:
          print(f"Error al agregar {current_id}: {e}")

      i += 1


### Reglas (documento PDF)

In [13]:
reglas_path = drive_path+'datos/Memoir_44_rules.pdf'
# Extraer texto completo del PDF
reglas_crudo = extract_text_with_pypdf2(reglas_path)
reglas = limpiar_texto(reglas_crudo)

In [14]:
reglas

'Rules & Battle ScenariosRules & Battle Scenarios For 2 players and team-playFor 2 players and team-playAges 8 and above 30-60 minutesAges 8 and above 30-60 minutes22Table of contents Components . . . . . . . . . . . . . . . . . . . 3 Setting up the Game . . . . . . . . . . . . 4Battlefield . . . . . . . . . . . . . . . . . . . . . 5Command Cards . . . . . . . . . . . . . . . . 5Object of the Game . . . . . . . . . . . . . . 6Game Turn . . . . . . . . . . . . . . . . . . . . . 6Play a Command Card . . . . . . . . . . .6Order Units . . . . . . . . . . . . . . . . . . . .6Move Units . . . . . . . . . . . . . . . . . . . .7 Infantry . . . . . . . . . . . . . . . . . . . . . .7Armor . . . . . . . . . . . . . . . . . . . . . . . .8Artillery . . . . . . . . . . . . . . . . . . . . . .8 Battle . . . . . . . . . . . . . . . . . . . . . . . . . 8 Battle Range . . . . . . . . . . . . . . . . . .9Line of Sight . . . . . . . . . . . . . . . . . .9Terrain Reductions . . . . . . . . . . . . .10 Reso

### Referencia principiantes (documento PDF)

In [15]:
referencia_principiantes_path = drive_path+"datos/Memoir_44_beginner's_reference.pdf"
referencia_principiantes_crudo = extract_text_with_pypdf2(referencia_principiantes_path)
referencia_principiantes = limpiar_texto(referencia_principiantes_crudo)

In [16]:
referencia_principiantes

'Days of Wonder, the Days of Wonder logo, Memoir ’44, the Memoir ’44 logotype and all related product and brand names are registered trademarks or trademarks of Days of Wonder, Inc. and copyrights © 2002 -2007 Days of Wonder, Inc. MEMOIR ’44 – A BEGINNER’S REFEREN CE INTRO This guide is targeted to Memoir ’44 beginners and is meant to help them better interpret some of the rules of the game. It is NOT meant to replace any of the following MUST READ documents : - the rulebook : (http://static.memoir44.com/lang/english/images/mm_rules_part1_en.pdf ) - the official FAQs: (http://www.days ofwonder.com/memoir44/en/content/faq/ ) - the unofficial FAQ available at BGG: (http://www.boardgamegeek.com/file/download/35704/M44_Unofficial_FAQ_v12.pdf ) The unoff icial FAQ is 90% similar to the official one, but better structured and easier to search for information. This document will not help more experienced players and also does not discuss (at least for now) any of the expansions, since the mai

### Escenarios (documento PDF)

In [17]:
escenariso_path = drive_path+"/datos/Memoir_44_scenarios.pdf"
escenarios_crudo = extract_text_with_pypdf2(escenariso_path)
escenarios = limpiar_texto(escenarios_crudo)

In [18]:
escenarios

'Pacific Theatre Scenario compilation Battles of Khalkhin-Gol ........................................ The Khalkhin-Gol Spark (Battle Map II, Vol. 1) ................................................. Kawatama Bridge (Battle Map II, Vol. 1) ................... Bain Tsagan Heights (Breakthrough) (Battle Map II, Vol. 1) ................................................. Soviet Infiltration (Battle Map II, Vol. 1) ...................................... The Halha River Shelling (Battle Map II, Vol. 1) .............................................. Down in the Dumps (Battle Map II, Vol. 1) ............................... Attack on the Soviet Outpost (Battle Map II, Vol. 1) .......... The Khalkhin-Gol Encirclement (Overlord) (Battle Map II, Vol. 1) ................ Tightening the Noose (Breakthrough) (Battle Map II, Vol. 1) Mara Shinko Sakusen ........................................................... Slim River (Scenario Book, Vol. II) ........................................................

### Tácticas (foro BGG)

In [52]:
url_tacticas = "https://boardgamegeek.com/thread/300195/tactics-for-memoir-44"
tacticas_crudo = extraer_comentarios_foro(url_tacticas)
tacticas = ''
for tactica_crudo in tacticas_crudo:
  tactica = limpiar_texto(tactica_crudo)
  tacticas += tactica
tacticas

'If I may offer my $.02 on the subject. These are the lessons I’ve learned in my first six games of Mem ’44 (note: haven’t played with artillery yet). There\'s a bit of luck, no question, with this card-driven order system, but there is also a good deal of strategy. - Let’s get this obvious one out of the way: bisectional units do it both ways. Given the choice, a unit on a split hex has options. OK, we all know this. - Wait for it... Take advantage of Airpower. Wait for your opponent to line up 4 units adjacent to each other and then thin them out with airpower. Unless you simply MUST play a Flank card, take advantage of the situation and use the Airpower card on no less than 3 units. - Buy yourself time: If most of your units are in one section, move your other units away from potential harm to a “safe” place and let them hide while you wait for the cards you need. - On that note, use the “Recon” card that allows you to draw 2 Command cards and choose the one you want. Even though yo

### Reseñas

#### Reseña 1 (foro BGG)

In [35]:
# Se espera la reseña más comentada del foro
url_hilo_reseña_1 = 'https://boardgamegeek.com/thread/210087/memoir-44-a-detailed-review'
hilo_reseña_1 = extraer_comentarios_foro(url_hilo_reseña_1)

In [36]:
len(hilo_reseña_1)

27

In [38]:
# Se aplica este método porque únicamente interesa el primer comentario que a su vez está dividido en varias de las etiquetas utilizadas para extraer los comentarios
reseña_1_crudo = ' '.join(hilo_reseña_1[:27])
reseña_1 = limpiar_texto(reseña_1_crudo)

In [39]:
reseña_1

"This review is the first detailed review I have attempted. I have tried to make this, part review and part resource, for anyone not totally familiar with the game. For this reason I expect readers to skip to the sections that are of most interest. Image Courtesy of Erich Summary Game Type - Board Game Play Time : 20-60 minutes Number of Players: 2-8 (Most commonly 2 players) Mechanics - Card Driven, Scenario Based, Dice Rolling Combat Difficulty - Moderate (Takes 6-12 plays to become totally familiar) Components - Very Good This is the first game I have revisited to do a more in depth analysis, as my first review was minimal to say the least. With over 200 plays of Memoir and the recent announcement of the Air Pack and further expansions, I felt the time was right to revisit the base game. Overview Memoir '44 is the 2nd game to be released in the 'Commands & Colours' System and like other games in the series Memoir is a scenario based board game. As the title suggests, the theme and s

#### Reseña 2 (blog Misut Meeple)

In [40]:
url_reseña_2 = "https://misutmeeple.com/2017/08/resena-memoir-44/"
reseña_2_crudo = extraer_reseña_blog(url_reseña_2)
reseña_2 = limpiar_texto(reseña_2_crudo)
reseña_2

/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'misutmeeple.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


'Hoy vamos a adentrarnos en un terreno poco explorado en este blog como es el de los wargames. Por mi perfil de jugador y el de mis diversos grupos, son diseños que no nos encajan demasiado por duración y complejidad (no tanto a nivel mecánico como de asimilación del juego en sí para sacarle provecho y no actuar de forma cuasi-aleatoria). Aquí es donde entra Richard Borg, autor del juego que hoy nos ocupa y, más concretamente, de un sistema de juego que estrenase con Battle Cry (wargame sobre la Guerra Civil Americana) que, de forma ultrasimplificada, acerca esta clase de juegos a los que, como yo, nos parecen una inversión de tiempo excesiva para la satisfacción obtenida.Empezaré justificando lo que acabo de soltar en el párrafo para que contextualicéis el tocho que se avecina. Partimos de la base de que la mayoría de los wargames requieren un mínimo de unas tres horas para desarrollar una partida (y de ahí para arriba), sin incluir la explicación, que suele implicar una buena dosis d

#### Reseña 3 (video YouTube)

In [41]:
# Se obtiene una lista de diccionarios que segmenta la transcripción
transcripcion_segmentada_crudo = YouTubeTranscriptApi.get_transcript('FUaYTE-qvK4')
# Cada diccionario tiene el atributo text con la porción del audio transcripto
reseña_3_crudo = ''
for segmento in transcripcion_segmentada_crudo:
  reseña_3_crudo += segmento['text']
reseña_3 = limpiar_texto(reseña_3_crudo)
reseña_3

"[Music]I got introduced into board games afterhaving a history in tabletop miniatureWar Games whilst I love the Miniaturesand the overall cosmetic appearanceminiature War Games provide the complexrules that most have always led me tospend more time painting or writing Armylist just about anything to do with thehobby but actually playing it so when aminiature war game board game with easymechanics Cor my attention I straightaway handed over my hard-earnedcash that's a lie after Consulting mysignificant other my beautifulsignificant other I handed over my hardeared ah our hard earned cash menoir 44by days of Wonder is a game for twoplayers or team play approximate playtime of 30 to 60 Minutes recommended forages eight and up Memoir 44 basicallyair drops you into the key battles toturn the tide in Western Europe duringWorld War to You'll command your troopsin battlefields that Define the historyof the 20th century the goal of the gameis dependent on the mission the playersare playing the

### Almacenaje de los textos

In [54]:
# Guardar información
pd_info = {
    'reglas' : reglas,
    'referencias_principiantes' : referencia_principiantes,
    'escenarios':escenarios,
    'tacticas': tacticas,
    'reseña_1_foro': reseña_1,
    'reseña_2_blog': reseña_2,
    'reseña_3_you_tube': reseña_3,

}

with open(drive_path+'/datos/pd_info.pkl', 'wb') as f:
    pickle.dump(pd_info, f)

### Carga de los textos

In [47]:
with open(drive_path+'/datos/pd_info.pkl', 'rb') as f:
    pd_info_loaded = pickle.load(f)

for seccion,texto in pd_info_loaded.items():
  print(f'----------{seccion}------')
  print(texto)
  print('\n\n')

----------reglas------
Rules & Battle ScenariosRules & Battle Scenarios For 2 players and team-playFor 2 players and team-playAges 8 and above 30-60 minutesAges 8 and above 30-60 minutes22Table of contents Components . . . . . . . . . . . . . . . . . . . 3 Setting up the Game . . . . . . . . . . . . 4Battlefield . . . . . . . . . . . . . . . . . . . . . 5Command Cards . . . . . . . . . . . . . . . . 5Object of the Game . . . . . . . . . . . . . . 6Game Turn . . . . . . . . . . . . . . . . . . . . . 6Play a Command Card . . . . . . . . . . .6Order Units . . . . . . . . . . . . . . . . . . . .6Move Units . . . . . . . . . . . . . . . . . . . .7 Infantry . . . . . . . . . . . . . . . . . . . . . .7Armor . . . . . . . . . . . . . . . . . . . . . . . .8Artillery . . . . . . . . . . . . . . . . . . . . . .8 Battle . . . . . . . . . . . . . . . . . . . . . . . . . 8 Battle Range . . . . . . . . . . . . . . . . . .9Line of Sight . . . . . . . . . . . . . . . . . .9Terrain Reductions . . . . . 

### Creación de base de dato vectorial junto a inserción de los textos

In [42]:
# Configurar el cliente de ChromaDB con persistencia
client = chromadb.Client(Settings(persist_directory=drive_path+"/bases_datos/vector_db"))

In [43]:
try:
  # Crear una colección llamada "vec_db"
  vec_db = client.create_collection("vec_db")
except:
  print('ya existe la base de datos vectorial')

In [44]:
#Eliminar la colección "vec_db" si corresponde
# try:
#   client.delete_collection("vec_db")
#   client.clear_system_cache()
# except ValueError:
#   print('no se puede eliminar porque aún no se creo la base de datos')

### Inserción de los textos en la base de datos vectorial

In [56]:
# Se trozan los textos para guardarlos en sus respectivas listas de chunk
chunk_reglas = chuncker(pd_info_loaded['reglas'])
chunk_referencias_principiantes = chuncker(pd_info_loaded['referencias_principiantes'])
chunk_escenarios = chuncker(pd_info_loaded['escenarios'])
chunk_referencias_tacticas = chuncker(pd_info_loaded['tacticas'])
chunk_reseña_1_foro = chuncker(pd_info_loaded['reseña_1_foro'])
chunk_reseña_2_blog = chuncker(pd_info_loaded['reseña_2_blog'])
chunk_reseña_3_you_tube = chuncker(pd_info_loaded['reseña_3_you_tube'])

In [57]:
insertar_db(chunk_reglas, {'seccion': 'reglas'}, 'reg')

ID: reg_0, Embedding Length: 512, Metadata: {'seccion': 'reglas'}, texto: Rules & Battle ScenariosRules & Battle Scenarios For 2 players and team-playFor 2 players and team-playAges 8 and above 30-60 minutesAges 8 and above 30-60 minutes22Table of contents Components . . . . . . . . . . . . . . . . . . . 3 Setting up the Game . . . . . . . . . . . . 4Battlefield . . . . . . . . . . . . . . . . . . . . . 5Command Cards . . . . . . . . . . . . . . . . 5Object of the Game
ID: reg_1, Embedding Length: 512, Metadata: {'seccion': 'reglas'}, texto: . . . . . 5Object of the Game . . . . . . . . . . . . . . 6Game Turn . . . . . . . . . . . . . . . . . . . . . 6Play a Command Card . . . . . . . . . . .6Order Units . . . . . . . . . . . . . . . . . . . .6Move Units . . . . . . . . . . . . . . . . . . . .7 Infantry . . . . . . . . . . . . . . . . . . . . . .7Armor . . . . . . . . . . . . . . . . . . . . . . . .8Artillery . . . . . . . . . . . .
ID: reg_2, Embedding Length: 512, Metadata: {'seccion

In [58]:
insertar_db(chunk_referencias_principiantes, {'seccion': 'referencias principales'}, 'ref_prin')

ID: ref_prin_0, Embedding Length: 512, Metadata: {'seccion': 'referencias principales'}, texto: Days of Wonder, the Days of Wonder logo, Memoir ’44, the Memoir ’44 logotype and all related product and brand names are registered trademarks or trademarks of Days of Wonder, Inc. and copyrights © 2002 -2007 Days of Wonder, Inc. MEMOIR ’44 – A BEGINNER’S REFEREN CE INTRO This guide is targeted to Memoir ’44 beginners and is meant to help them better interpret some of the rules of the game. It is
ID: ref_prin_1, Embedding Length: 512, Metadata: {'seccion': 'referencias principales'}, texto: the rules of the game. It is NOT meant to replace any of the following MUST READ documents : - the rulebook : (http://static.memoir44.com/lang/english/images/mm_rules_part1_en.pdf ) - the official FAQs: (http://www.days ofwonder.com/memoir44/en/content/faq/ ) - the unofficial FAQ available at BGG: (http://www.boardgamegeek.com/file/download/35704/M44_Unofficial_FAQ_v12.pdf ) The unoff icial FAQ
ID: ref_pr

In [59]:
insertar_db(chunk_escenarios, {'seccion': 'escenarios'}, 'esc')

ID: esc_0, Embedding Length: 512, Metadata: {'seccion': 'escenarios'}, texto: Pacific Theatre Scenario compilation Battles of Khalkhin-Gol ........................................ The Khalkhin-Gol Spark (Battle Map II, Vol. 1) ................................................. Kawatama Bridge (Battle Map II, Vol. 1) ................... Bain Tsagan Heights (Breakthrough) (Battle Map II, Vol. 1) ................................................. Soviet Infiltration (Battle Map
ID: esc_1, Embedding Length: 512, Metadata: {'seccion': 'escenarios'}, texto: Infiltration (Battle Map II, Vol. 1) ...................................... The Halha River Shelling (Battle Map II, Vol. 1) .............................................. Down in the Dumps (Battle Map II, Vol. 1) ............................... Attack on the Soviet Outpost (Battle Map II, Vol. 1) .......... The Khalkhin-Gol Encirclement (Overlord) (Battle Map II, Vol. 1) ................
ID: esc_2, Embedding Length: 512, Metadata: {'seccio

In [60]:
insertar_db(chunk_reseña_1_foro, {'seccion': 'reseña 1 foro'}, 'res_1_f')

ID: res_1_f_0, Embedding Length: 512, Metadata: {'seccion': 'reseña 1 foro'}, texto: This review is the first detailed review I have attempted. I have tried to make this, part review and part resource, for anyone not totally familiar with the game. For this reason I expect readers to skip to the sections that are of most interest. Image Courtesy of Erich Summary Game Type - Board Game Play Time : 20-60 minutes Number of Players: 2-8 (Most commonly 2 players) Mechanics - Card
ID: res_1_f_1, Embedding Length: 512, Metadata: {'seccion': 'reseña 1 foro'}, texto: 2 players) Mechanics - Card Driven, Scenario Based, Dice Rolling Combat Difficulty - Moderate (Takes 6-12 plays to become totally familiar) Components - Very Good This is the first game I have revisited to do a more in depth analysis, as my first review was minimal to say the least. With over 200 plays of Memoir and the recent announcement of the Air Pack and further expansions, I felt the time
ID: res_1_f_2, Embedding Length: 512,

In [61]:
insertar_db(chunk_reseña_2_blog, {'seccion': 'reseña 2 blog'}, 'res_2_b')

ID: res_2_b_0, Embedding Length: 512, Metadata: {'seccion': 'reseña 2 blog'}, texto: Today we are going to delve into a little-explored area on this blog, which is wargames. Given my profile as a player and that of my various groups, these are designs that do not fit us very well in terms of duration and complexity (not so much on a mechanical level as in terms of assimilating the game itself to take advantage of it and not act in a quasi-random way). This is where Richard Borg comes in, author of the game that today
ID: res_2_b_1, Embedding Length: 512, Metadata: {'seccion': 'reseña 2 blog'}, texto: Borg, author of the game we are dealing with today and, more specifically, of a game system that he debuted with Battle Cry (a wargame about the American Civil War) that, in an ultra-simplified way, brings this kind of game closer to those who, like me, find it an excessive investment of time for the satisfaction obtained. I will start by justifying what I just said in the paragraph so tha

In [62]:
insertar_db(chunk_reseña_3_you_tube, {'seccion': 'reseña 3 you tube'}, 'res_3_y')

ID: res_3_y_0, Embedding Length: 512, Metadata: {'seccion': 'reseña 3 you tube'}, texto: [Music]I got introduced into board games afterhaving a history in tabletop miniatureWar Games whilst I love the Miniaturesand the overall cosmetic appearanceminiature War Games provide the complexrules that most have always led me tospend more time painting or writing Armylist just about anything to do with thehobby but actually playing it so when aminiature war game board game with easymechanics
ID: res_3_y_1, Embedding Length: 512, Metadata: {'seccion': 'reseña 3 you tube'}, texto: board game with easymechanics Cor my attention I straightaway handed over my hard-earnedcash that's a lie after Consulting mysignificant other my beautifulsignificant other I handed over my hardeared ah our hard earned cash menoir 44by days of Wonder is a game for twoplayers or team play approximate playtime of 30 to 60 Minutes recommended forages eight and up Memoir 44 basicallyair drops you
ID: res_3_y_2, Embedding L

### Acceso a la base de datos vectorial

In [63]:
# Acceder a la colección existente
vec_db = client.get_collection("vec_db")

In [64]:
# Prueba
consulta = "¿es un juego interesante?"
embedding_consulta = embed([consulta]).numpy().tolist()
results = vec_db.query(
    query_embeddings=embedding_consulta,  # Aquí pasamos el embedding de la consulta
    n_results=5,

)
results['documents']

[["up the game You will greatly benefit by applying a small degree of discipline (Buck Up Soldier!) when setting up a game scenario in Memoir '44. We strongly recommend that you follow the step-by-step approach outlined below when setting up a game, especially for your first few plays. The reward is an endless variety of battlefields to play on, and theopportunity to pick-up fascinating historical",
  'much further. To establish a similar case, Memoir ’44 would be the Stone Age (here is its long review) of worker placement games. It serves perfectly to see if the design fits your tastes, but many will not want to go further and will be satisfied with the Stone Age game. As you can imagine, the replayability is more than remarkable as the basic game includes a total of 16',
  'Daily game of about an hour and a half at most and some sessions during the weekend, but nothing particularly spectacular (4-5 hours at most). So, the chances of fitting a game of this type, just by duration, are 

## Parte 2: Base de datos de grafos

La base de datos de grafos se utiliza para modelar datos acerca del juego que no tienen que ver de manera estricta con mecánica y que no requieren amplia explicación. Se utiliza principalmente para conocer quién son las personas que
participaron en su creación y con cuáles otros juegos están vinculados estas
personas.

### Funciones

In [65]:
def analizar_creditos(url_creditos)->list:

    '''
    devuelve los enlaces presentes en los créditos del juego extrayendo
    el valor del atributo 'ng-href' para cada elemento a que se encuentre
    '''
    # Configurar las opciones del navegador
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--disable-gpu')

    # Configurar la ruta del driver
    service = Service('/usr/bin/chromedriver')

    driver = webdriver.Chrome(options=chrome_options)
    #driver.get('chrome://settings/clearBrowserData') comento porque arroja error invalid session ID
    try:
        driver.get(url_creditos)
        elementos_a = driver.find_elements(By.TAG_NAME,'a')
        enlaces = [elemento_a.get_attribute('ng-href') for elemento_a in elementos_a]
        #
        textos_span_especificos = [elemento.text for elemento in driver.find_elements(By.XPATH, "//span[@ng-switch='info.fieldtype']")]
        año_lanzamiento = [texto for texto in textos_span_especificos if texto.isdigit()][0]

        return enlaces,año_lanzamiento

    finally:
        # Cerrar el navegador
        driver.quit()

In [66]:
def filtrar_enlaces_relevantes(links:list,campos_interes:list)->list[tuple]:
  '''
  A partir de una lista de enlaces y otro lista con etiquetas relevantes
  devuelve los enlaces de interés con sus respectivas etiquetas
  '''
  enlaces_relevantes = []
  for enlace in enlaces:
    # boardgamemechanic/2676/grid-movement
    if not isinstance(enlace,str): continue
    resultado = re.search(r'/boardgame.*/.*/*',enlace)
    if resultado is None:
      continue
    else:
      for campo_relevante in campos_relevantes:
        if campo_relevante in enlace:
          enlaces_relevantes.append((enlace,campo_relevante))
  return enlaces_relevantes


In [67]:
def determinar_predicado_segun_rol(rol:str)->tuple:
  """
  Según el rol de la entidad se determina el predicado tanto para el primero
  como para el segundo nivel
  """
  if rol == 'designer':
    return 'diseñado_por','diseñó'
  elif rol == 'artist':
    return 'tiene_como_artista' ,'participó_en'
  elif rol == 'publisher':
    return 'publicado_por','publicó'
  elif rol == 'mechanic':
    return 'utiliza_mecanismo','utilizado_por'

In [68]:
def extraer_entidad(url_entidad:str)->tuple:
    '''
    A partir de la url de la entidad extrae su nombre
    junto a sus juegos asociados
    '''
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--disable-gpu')

    # Configurar la ruta del driver
    service = Service('/usr/bin/chromedriver')

    # comento porque arroja error invalid session ID
    driver = webdriver.Chrome(options=chrome_options)
    try:
        driver.get('https://boardgamegeek.com'+url_entidad)
        # Primer nivel del grafo
        nombre_sucio_entidad = url_entidad.split('/')[-1]
        buscador = f'a.ng-binding[ng-href*="{nombre_sucio_entidad}"]'
        links = driver.find_element(By.CSS_SELECTOR, buscador)
        nombre_limpio_entidad = links.text.strip()
        # segundo nivel del grafo
        elementos_juegos_asociados = driver.find_elements(By.CLASS_NAME,'rec-title')
        juegos_asociados = [elemento_juego_asociado.text for elemento_juego_asociado in elementos_juegos_asociados]
        return nombre_limpio_entidad,juegos_asociados
    finally:
        # Cerrar el navegador
        driver.quit()

### Extracción de la información

In [69]:
url_creditos = "https://boardgamegeek.com/boardgame/10630/memoir-44/credits"
enlaces,año_lanzamiento = analizar_creditos(url_creditos)
campos_relevantes = ['designer','artist','publisher','mechanic']
info_enlaces_relevantes = filtrar_enlaces_relevantes(enlaces,campos_relevantes)

In [70]:
ontologia = []
for enlace,rol in info_enlaces_relevantes:
  # primer nivel
  triada_1_nivel = {}
  triada_1_nivel['sujeto'] = "Memoir 44"
  triada_1_nivel['tipo_sujeto'] = "Game"
  triada_1_nivel['predicado'] = determinar_predicado_segun_rol(rol)[0]
  nombre_entidad,juegos_asociados = extraer_entidad(enlace)
  triada_1_nivel['objeto'] = nombre_entidad
  triada_1_nivel['tipo_objeto'] = rol.capitalize()
  ontologia.append(triada_1_nivel)
  # segundo nivel
  for juego_asociado in juegos_asociados:
    triada_2_nivel = {}
    triada_2_nivel['sujeto'] = nombre_entidad
    triada_2_nivel['tipo_sujeto'] = rol.capitalize()
    triada_2_nivel['predicado'] = determinar_predicado_segun_rol(rol)[1]
    triada_2_nivel['objeto'] = juego_asociado
    triada_2_nivel['tipo_objeto'] = 'Game'
    ontologia.append(triada_2_nivel)

# Agrego la triada vinculada con el año de lanzamiento
triada_año_lanzamiento = {
  'sujeto' : "Memoir 44",
  'tipo_sujeto' : "Game",
  'predicado' : 'lanzado_en',
  'objeto' : año_lanzamiento,
  'tipo_objeto' : 'Year'
}
ontologia.append(triada_año_lanzamiento)


In [71]:
ontologia

[{'sujeto': 'Memoir 44',
  'tipo_sujeto': 'Game',
  'predicado': 'diseñado_por',
  'objeto': 'Richard Borg',
  'tipo_objeto': 'Designer'},
 {'sujeto': 'Richard Borg',
  'tipo_sujeto': 'Designer',
  'predicado': 'diseñó',
  'objeto': "Memoir '44",
  'tipo_objeto': 'Game'},
 {'sujeto': 'Richard Borg',
  'tipo_sujeto': 'Designer',
  'predicado': 'diseñó',
  'objeto': 'Commands & Colors: Ancients',
  'tipo_objeto': 'Game'},
 {'sujeto': 'Richard Borg',
  'tipo_sujeto': 'Designer',
  'predicado': 'diseñó',
  'objeto': 'BattleLore: Second Edition',
  'tipo_objeto': 'Game'},
 {'sujeto': 'Richard Borg',
  'tipo_sujeto': 'Designer',
  'predicado': 'diseñó',
  'objeto': 'Commands & Colors: Napoleonics',
  'tipo_objeto': 'Game'},
 {'sujeto': 'Richard Borg',
  'tipo_sujeto': 'Designer',
  'predicado': 'diseñó',
  'objeto': 'BattleLore',
  'tipo_objeto': 'Game'},
 {'sujeto': 'Richard Borg',
  'tipo_sujeto': 'Designer',
  'predicado': 'diseñó',
  'objeto': 'Perudo',
  'tipo_objeto': 'Game'},
 {'sujet

### Armado de base de datos

#### Crear el grafo base

In [72]:
G = nx.DiGraph()
for triada in ontologia:
  sujeto,tipo_sujeto,predicado,objeto,tipo_objeto = triada.values()
  G.add_node(sujeto,type=tipo_sujeto)
  G.add_node(objeto,type=tipo_objeto)
  G.add_edge(sujeto,objeto,relationship=predicado)

In [73]:
print('la cantidad de nodos que contiene el grafo es ',len(G.nodes))

la cantidad de nodos que contiene el grafo es  84


In [74]:
# Generar posiciones de los nodos con un layout de NetworkX
pos = nx.spring_layout(G, seed=42)  # Layout basado en fuerzas para distribuir los nodos

# Extraer las coordenadas de los nodos
x_nodes = [pos[node][0] for node in G.nodes]
y_nodes = [pos[node][1] for node in G.nodes]

# Extraer las relaciones (aristas)
edge_x = []
edge_y = []
edge_labels = []  # Aquí guardaremos las etiquetas de las aristas
edge_midpoints = []  # Para almacenar las posiciones medias de las aristas
for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x.append(x0)
    edge_x.append(x1)
    edge_y.append(y0)
    edge_y.append(y1)

    # Guardamos la relación como etiqueta
    edge_labels.append(G[edge[0]][edge[1]]['relationship'])

    # Calculamos los puntos medios de las aristas
    mid_x = (x0 + x1) / 2
    mid_y = (y0 + y1) / 2
    edge_midpoints.append((mid_x, mid_y))

# Crear el gráfico interactivo con Plotly
edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='gray'),
    hoverinfo='none', mode='lines')

node_trace = go.Scatter(
    x=x_nodes, y=y_nodes,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        colorscale='YlGnBu',
        size=20,
        colorbar=dict(thickness=15, title='Node Connections', xanchor='left', titleside='right')
    ))

# Añadir información sobre los nodos (nombre, tipo, etc.)
node_text = []
for node in G.nodes():
    node_info = f'{node} ({G.nodes[node].get("type", "unknown")})'
    node_text.append(node_info)

node_trace.marker.color = np.arange(len(G.nodes))  # Colores por nodo
node_trace.text = node_text  # Información que aparece al pasar el ratón por encima

# Añadir las etiquetas de las relaciones (aristas) en el punto medio de cada arista
edge_trace_text = go.Scatter(
    x=[mid[0] for mid in edge_midpoints], y=[mid[1] for mid in edge_midpoints],
    mode='text',
    text=edge_labels,
    textposition='middle center',
    hoverinfo='none',
    showlegend=False,
    textfont=dict(size=10, color="black", family="Arial")
)

# Crear el layout interactivo
layout = go.Layout(
    title="Interactive Network Graph of Memoir '44",
    titlefont_size=16,
    showlegend=False,
    hovermode='closest',
    xaxis=dict(showgrid=False, zeroline=False),
    yaxis=dict(showgrid=False, zeroline=False),
    plot_bgcolor='white'
)
# Añadir las etiquetas de los nodos (nombre de la entidad) sobre los nodos
node_trace_text = go.Scatter(
    x=x_nodes, y=[y + 0.05 for y in y_nodes],  # Ajuste de posición para que el texto esté encima de los nodos
    mode='text',
    text=node_text,
    textposition='bottom center',  # Asegura que el texto esté por encima del nodo
    hoverinfo='none',
    showlegend=False,
    textfont=dict(size=10, color="black", family="Arial")
)

# Crear la figura de Plotly con los cambios
fig = go.Figure(data=[edge_trace, node_trace, edge_trace_text, node_trace_text], layout=layout)

# Mostrar el gráfico interactivo
fig.show()

#### Crear una base de datos RDF

In [75]:
# Crear un grafo RDF vacío
g = rdflib.Graph()

# Definir los prefijos
ex = rdflib.Namespace("http://memoir_44.org/")
g.bind("ex", ex)

# Función para codificar los nombres a formato URI válido
def encode_to_uri(name):
    return urllib.parse.quote(name.replace(" ", "_"))

# Se utilizan estas dos colecciones para pasárselas al LLM encargado de generar
# las consultas SPARQL
entidades = set()
relaciones = []

for triada in ontologia:
  sujeto,tipo_sujeto,predicado,objeto,tipo_objeto = triada.values()
  nodo_sujeto = encode_to_uri(sujeto)
  nodo_objeto = encode_to_uri(objeto)
  entidades.add(nodo_sujeto)
  entidades.add(nodo_objeto)
  relacion = (nodo_sujeto,predicado,nodo_objeto)
  relaciones.append(relacion)
  g.add((ex[nodo_sujeto], rdflib.RDF.type, ex[tipo_sujeto]))
  g.add((ex[nodo_objeto], rdflib.RDF.type, ex[tipo_objeto]))
  g.add((ex[nodo_sujeto], ex[predicado], ex[nodo_objeto]))

#Serializar y exportar el grafo a RDF/XML
rdf_output = g.serialize(format='xml')
print(rdf_output)

# Guardar el RDF en un archivo
with open(drive_path+'bases_datos/graph_db.rdf', "w") as file:
    file.write(rdf_output)


<?xml version="1.0" encoding="utf-8"?>
<rdf:RDF
   xmlns:ex="http://memoir_44.org/"
   xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"
>
  <rdf:Description rdf:about="http://memoir_44.org/Scenario_/_Mission_/_Campaign_Game">
    <rdf:type rdf:resource="http://memoir_44.org/Mechanic"/>
    <ex:utilizado_por rdf:resource="http://memoir_44.org/Pandemic_Legacy%3A_Season_1"/>
    <ex:utilizado_por rdf:resource="http://memoir_44.org/Gloomhaven"/>
    <ex:utilizado_por rdf:resource="http://memoir_44.org/Gloomhaven%3A_Jaws_of_the_Lion"/>
    <ex:utilizado_por rdf:resource="http://memoir_44.org/Clank%21_Legacy%3A_Acquisitions_Incorporated"/>
    <ex:utilizado_por rdf:resource="http://memoir_44.org/Frosthaven"/>
    <ex:utilizado_por rdf:resource="http://memoir_44.org/Arkham_Horror%3A_The_Card_Game"/>
  </rdf:Description>
  <rdf:Description rdf:about="http://memoir_44.org/Claude_Rica">
    <rdf:type rdf:resource="http://memoir_44.org/Artist"/>
    <ex:participó_en rdf:resource="http://me

In [76]:
def get_triadas(input_text: str,texto:str):
    """
    Extraer tríadas de un texto.
    """
    # La API de Hugging Face
    api_key = config('HUGGINGFACE_TOKEN', userdata.get('HUGGINGFACE_TOKEN'))
    client_HF = InferenceClient(api_key= api_key)

    chat_prompt = [{
        "role": "system",
        "content": f"""
        Eres un modelo que extrae triadas de un texto a partir de identificar las entidades y las relaciones
        presentes en el mismo.
        Estas triadas van a ser luego insertadas en una basa de datos de grafos RDF.
        Una triada tiene los siguientes elementos: sujeto, predicado y objeto.
        Es importante que sólo respondas en base a la siguiente información: {texto}.
        Tienes que devolver nodos junto a sus conexiones.
        """
    }, {
        "role": "user",
        "content": input_text
    }]

    # Realizar la solicitud de completado de chat usando el cliente
    completion = client_HF.chat.completions.create(
        model="HuggingFaceH4/zephyr-7b-beta",
        messages=chat_prompt,
        max_tokens=50,
        temperature=0.1
    )

    # Extraer y devolver la consulta SPARQL generada
    return completion.choices[0].message['content']

In [77]:
texto = """
Memoir '44 is a historical boardgame where players face-off in stylized battles of some of the most famous historic battles of World War II including Omaha Beach, Pegasus Bridge, Operation Cobra and the Ardennes.
Memoir '44 includes over 15 different battle scenarios and features a double-sided hex game board for both beach landings and countryside combat. Each scenario mimics the historical terrain, troop placements and objectives of each army. Commanders deploy troops through Command and Tactic cards, applying the unique skills of his units -- infantry, paratrooper, tank, artillery, and even resistance fighters -- to their greatest strength.
"By design, the game is not overly complex", says Memoir '44 designer, Richard Borg. "The game mechanics, although simple, still require strategic card play, timely dice rolling and an aggressive yet flexible battle plan to achieve victory." In addition to the large, double-sided gameboard, Memoir '44 includes 144 amazingly detailed army miniatures - including historically accurate infantry, tanks and artillery; 36 Obstacle pieces, 60 illustrated Command cards, 44 Special Terrain tiles, and 8 Custom Wooden dice.
Memoir '44 is designed for 2 players but easily accommodates team play. With Memoir '44 Overlord scenarios, players can use multiple boards and up to 8 players to conduct large scale operations, experiencing the challenges of troop coordination and military chain of command on a large scale battlefield. Average game length is between 30 and 60 minutes, encouraging match play where players can command first one side and then the other.
The Memoir '44 series consists of the base game and a number of expansions.
This game is based upon Richard Borg's Command and Colors system.
"""
get_triadas('¿Cuáles entidades y relaciones identificas en el texto?',texto)

"Entidades:\n1. Memoir '44 (juego histórico de mesa)\n2. Jugadores (dos o más)\n3. Escenarios de batalla (más de 15)\n"

### Carga de la base de datos de grafo

In [78]:
# Cargar el archivo RDF
from rdflib import Graph
rdf_file = drive_path+'bases_datos/graph_db.rdf'  # Reemplaza con la ruta a tu archivo
g = Graph()
# Parsear el archivo para construir el grafo
g.parse(rdf_file, format="xml")  # Cambia el formato si no es RDF/XML

<Graph identifier=N8eb5ea34d125402aa50a28e71adc1cc6 (<class 'rdflib.graph.Graph'>)>

In [79]:
# Se prueba una query relativa al diseñador del juego
query = """
    SELECT (STRAFTER(STR(?Artist), "http://memoir_44.org/") AS ?name)
    WHERE {{
        ?game a <http://memoir_44.org/Game> .
        ?game <http://memoir_44.org/tiene_como_artista> ?Artist .
    }}
"""

# Ejecutar consulta y mostrar resultados
results = g.query(query)
for row in results:
    print(row.name)

Cyrille_Daujean
Julien_Delval
Don_Perrin
Claude_Rica


In [80]:
query = """
SELECT (STRAFTER(STR(?Year), 'http://memoir_44.org/') AS ?name)
WHERE {
    ?game a <http://memoir_44.org/Game> .
    ?game <http://memoir_44.org/lanzado_en> ?Year .
  }
"""
# Ejecutar consulta y mostrar resultados
results = g.query(query)
for row in results:
    print(row.name)

2004


##Parte 3: Base Tabular
La base tabular tendrá como objetivo almacenar:

* Datos estadísticos: jugadores totales, por mes, etc.
* Dificultad del juego.
* Cantidad de fichas.
* Precio promedio entre sitios web.
* Tiempo de juego
* Edad recomendada.
* Puntuación.

#### Funciones

In [81]:
def obtener_estadisticas_y_valores(url:str)->list:
    '''
    Obtiene las principales estadísticas del juego, las cuales las devuelve
    a través de una lista con información sobre la estadística en cuestión y su
    respectivo valor
    '''

    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--disable-gpu')

    # Configurar la ruta del driver
    service = Service('/usr/bin/chromedriver')

    driver = webdriver.Chrome(options=chrome_options)

    driver.get(url)
    # Crear una lista para almacenar los resultados
    resultados = []

    # Espera explícita para asegurarse de que la página cargue completamente
    try:
        # Espera a que los elementos <li> con la clase "outline-item" sean visibles
        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'li.outline-item'))
        )

        # Encuentra todos los elementos <li> con la clase "outline-item"
        items = driver.find_elements(By.CSS_SELECTOR, 'li.outline-item')

        # Itera sobre los elementos y extrae la entidad y el valor
        for item in items:
            try:
                # Encuentra la entidad (titulo) dentro del div con clase "outline-item-title"
                entidad = item.find_element(By.CSS_SELECTOR, 'div.outline-item-title').text.strip()

                if entidad == 'All time Plays':
                    entidad ='Total of players'
                elif entidad == 'This Month':
                    entidad ='Players this Month'
                elif entidad == 'Own':
                    entidad ='Players who own the game'
                elif entidad == 'Prev. Owned':
                    entidad ='Players who owned the game before but not now'
                elif entidad == 'For Trade':
                    entidad ='Players who have the game for trade'
                elif entidad == 'Want In Trade':
                    entidad ='Players who want the game in trade'
                elif entidad == 'Wishlist':
                    entidad ='Players who have the game in their wishlist'
                elif entidad == 'Has Parts':
                    entidad ='Players who have parts of the game'
                elif entidad == 'Want Parts':
                    entidad ='Players who want parts of the game'

                # Encuentra el valor (rating) dentro del div con clase "outline-item-description"
                valor = item.find_element(By.CSS_SELECTOR, 'div.outline-item-description a').text.strip()

                # Almacena el resultado en el formato 'Entidad: Valor'
                resultados.append({'Entidad': entidad, 'Valor': valor})
            except Exception as e:
                # Si ocurre un error al extraer los datos de una entidad, lo omite
                pass
    except Exception as e:
        print(f"Error al esperar elementos: {e}")

    # Cierra el navegador
    driver.quit()

    # Devuelve la lista de resultados
    return resultados

In [82]:
ruta_estadisticas_csv = drive_path+'/bases_datos/tabular_db.csv'

### Extracción de datos y creación de la base de datos tabular

In [83]:
url_estadisticas = 'https://boardgamegeek.com/boardgame/10630/memoir-44/stats'
estadisticas = obtener_estadisticas_y_valores(url_estadisticas)
estadisticas

[{'Entidad': 'Avg. Rating', 'Valor': '7.532'},
 {'Entidad': 'No. of Ratings', 'Valor': '30,811'},
 {'Entidad': 'Weight', 'Valor': '2.27 / 5'},
 {'Entidad': 'Comments', 'Valor': '7,277'},
 {'Entidad': 'Fans', 'Valor': '2,301'},
 {'Entidad': 'Overall Rank', 'Valor': '205'},
 {'Entidad': 'War Rank', 'Valor': '68'},
 {'Entidad': 'All Time Plays', 'Valor': '172,716'},
 {'Entidad': 'Players this Month', 'Valor': '310'},
 {'Entidad': 'Players who own the game', 'Valor': '41,402'},
 {'Entidad': 'Players who owned the game before but not now',
  'Valor': '5,602'},
 {'Entidad': 'Players who have the game for trade', 'Valor': '769'},
 {'Entidad': 'Players who want the game in trade', 'Valor': '936'},
 {'Entidad': 'Players who have the game in their wishlist', 'Valor': '6,551'},
 {'Entidad': 'Players who have parts of the game', 'Valor': '25'},
 {'Entidad': 'Players who want parts of the game', 'Valor': '51'}]

In [84]:
df_estadisticas = pd.DataFrame(estadisticas)
df_estadisticas.to_csv(ruta_estadisticas_csv,index=False)

### Carga de la base de datos tabular

In [85]:
df_estadisticas = pd.read_csv(ruta_estadisticas_csv)
df_estadisticas

,Entidad,Valor
0,Avg. Rating,7.532
1,No. of Ratings,"30,811"
2,Weight,2.27 / 5
3,Comments,"7,277"
4,Fans,"2,301"
5,Overall Rank,205
6,War Rank,68
7,All Time Plays,"172,716"
8,Players this Month,310
9,Players who own the game,"41,402"


##Parte 4: Modelos clasificadores

In [86]:
# Lista de clases de categorías
clases = ['tabular_db', 'vector_db', 'graph_db']

### Modelo Regresión Logística

In [87]:
dataset_clasificador = pd.read_excel(drive_path+'datos/dataset_clasificador.xlsx')
dataset_clasificador.head()

,consulta,clasificación
0,¿Cuántas personas tienen el juego disponible p...,tabular_db
1,¿Cuántos jugadores desean obtener el juego med...,tabular_db
2,¿Cuántos jugadores tienen el juego en su lista...,tabular_db
3,¿Es posible ver cuántos jugadores tienen parte...,tabular_db
4,¿Cuántos jugadores están buscando partes del j...,tabular_db


In [88]:
dataset_clasificador['clasificación'].unique()

array(['tabular_db', 'vector_db', 'graph_db'], dtype=object)

In [89]:
X = dataset_clasificador['consulta']
y = dataset_clasificador['clasificación']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [90]:
X_train_embed = X_train.apply(lambda x: embed(x).numpy().flatten())

X_test_embed =  X_test.apply(lambda x: embed(x).numpy().flatten())
X_train_embed = np.array(X_train_embed.tolist())
X_test_embed =  np.array(X_test_embed.tolist())

In [91]:
# Entrenar un clasificador como Logistic Regression
clasificador_reg_log = LogisticRegression(max_iter=1000, class_weight='balanced')
clasificador_reg_log.fit(X_train_embed, y_train)

LogisticRegression(class_weight='balanced', max_iter=1000)

In [92]:
y_pred = clasificador_reg_log.predict(X_test_embed)


In [93]:
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

    graph_db       0.93      0.93      0.93        15
  tabular_db       0.80      0.80      0.80         5
   vector_db       0.88      0.88      0.88        16

    accuracy                           0.89        36
   macro avg       0.87      0.87      0.87        36
weighted avg       0.89      0.89      0.89        36



Se puede observar que se tiene un f1-score relativamente aceptable para las clases, aunque se vislumbra un sesgo hacia el error para la clase tabular_db.

###Modelo LLM.

In [94]:
def clasificador_llm_predict(frase: str)->str:
    """
    En base a una consulta predice a cuál categoría corresponde. Cada categoría
    está asociada a una base de datos en particular.
    """
    # Clave API de Hugging Face
    api_key = config('HUGGINGFACE_TOKEN', userdata.get('HUGGINGFACE_TOKEN'))
    client_HF = InferenceClient(api_key= api_key)

   # Prompt para clasificación
    chat_prompt = [{
        "role": "system",
        "content": f"Eres un modelo de clasificación. La tarea es clasificar frases en una de estas categorías: [{', '.join(clases)}]."
                   " Dada una frase debes solo escribir la categoria clasificada sin escribir la respuesta en oración."
                   " Tienes prohibido dar clasificaciones por fuera de las clases mencionadas o escribir palabras que no sean las de las clases"
                   "Si la frase es sobre el reglamento, las estrategias u opiniones sobre el juego debes debes devolver: vector_db"
                   "Si la frase es sobre personas que participaron en la realización del juego debes debes devolver: graph_db"
                   "Si la frase es sobre las estadísticas y números del juego debes debes devolver: tabular_db"
                   "Por ejemplo, si el usuario escribe 'Como puedo hacer para ganar?' debes devolver: vector_db"
                   "Otro ejemplo: si el usuario escribe 'En qué año se hizo el juego? debes devolver: graph_db"
                   "Por ejemplo si el usuario escribe 'Qué puntaje tiene el juego?' debes devolver: tabular_db"
    }, {
        "role": "user",
        "content": frase
    }]

     # Realizar la solicitud de completado de chat usando el cliente
    completion = client_HF.chat.completions.create(
        model="Qwen/Qwen2.5-72B-Instruct",
        messages=chat_prompt,
        max_tokens=50,
        temperature=0.1
    )


    # Imprimir solo la clasificación sin contexto adicional
    return completion.choices[0].message['content']

### Comparación de modelos de clasificación

In [95]:
test_consultas = [
  ('Cuál estrategia puedo seguir para ganar el juego?', 'vector_db'),
  ('¿Cuál empresa comercializa el juego?', 'graph_db'),
  ('¿Cómo comenzar a jugar?', 'vector_db'),
  ('¿quién diseñó el juego?', 'graph_db'),
  ('¿Qué tan ponderado es el juego?', 'tabular_db'),
  ('Cuál estrategia puedo implementar para jugar?', 'vector_db'),
  ('quienes crearon el juego?', 'graph_db'),
  ('Qué puntaje tiene el juego?', 'tabular_db'),
  ('¿Cuál es la posición del juego en el ranking?', 'tabular_db')
]


In [96]:
errores_test_clasificador_reg_log = 0
errores_test_clasificador_llm = 0
for consulta,etiqueta_real in test_consultas:
    etiqueta_predicha_reg_log = clasificador_reg_log.predict(embed([consulta]))[0]
    etiqueta_predicha_llm = clasificador_llm_predict(consulta)
    errores_test_clasificador_reg_log += 1 if etiqueta_predicha_reg_log != etiqueta_real else 0
    errores_test_clasificador_llm += 1 if etiqueta_predicha_llm != etiqueta_real else 0

In [97]:
print(f'Errores en test según modelo de clasificación:\n\t- regresión logística: {errores_test_clasificador_reg_log}\n\t- llm: {errores_test_clasificador_llm}')


Errores en test según modelo de clasificación:
	- regresión logística: 1
	- llm: 0


Se puede ver un mejor rendimiento del llm que de la regresión logística por lo que se utilizara este en adelante para clasificar la consulta del usuario a la hora de determinar a cuál fuente de datos redirigir la consulta.

##Parte 5: Obtención de query según consulta

### Query para base de dato vectorial

In [98]:
lista_chunks_vec_db = vec_db.get()['documents']

In [99]:
lista_chunks_vec_db

['Rules & Battle ScenariosRules & Battle Scenarios For 2 players and team-playFor 2 players and team-playAges 8 and above 30-60 minutesAges 8 and above 30-60 minutes22Table of contents Components . . . . . . . . . . . . . . . . . . . 3 Setting up the Game . . . . . . . . . . . . 4Battlefield . . . . . . . . . . . . . . . . . . . . . 5Command Cards . . . . . . . . . . . . . . . . 5Object of the Game',
 '. . . . . 5Object of the Game . . . . . . . . . . . . . . 6Game Turn . . . . . . . . . . . . . . . . . . . . . 6Play a Command Card . . . . . . . . . . .6Order Units . . . . . . . . . . . . . . . . . . . .6Move Units . . . . . . . . . . . . . . . . . . . .7 Infantry . . . . . . . . . . . . . . . . . . . . . .7Armor . . . . . . . . . . . . . . . . . . . . . . . .8Artillery . . . . . . . . . . . .',
 '. . . . . . . . . . . . . . . . . . . . . .8 Battle . . . . . . . . . . . . . . . . . . . . . . . . . 8 Battle Range . . . . . . . . . . . . . . . . . .9Line of Sight . . . . . . . . . . . . 

In [100]:
# Función para tokenizar
def tokenize(chunks: list[str]) -> list[list[str]]:
    """
    Tokeniza una lista de textos dividiendo cada texto en palabras individuales
    y filtrando los caracteres no alfanuméricos.
    """
    tokenized_chunks = []

    for text in chunks:
        tokens = word_tokenize(text)
        tokenized_chunks.append([word.lower() for word in tokens if word.isalnum()])

    return tokenized_chunks

In [101]:
# Función para obtener los mejores resultados de BM25
def busqueda_palabras_claves(query_text: str, top_n: int = 5) -> list[tuple]:
    """
    En base a una consulta busca los documentos más relevantes utilizando el
    modelo BM25. Retorta los documentos recuperados junto a sus
    respectivos puntajes.
    """

    # Crear el modelo BM25
    tokenized_docs = tokenize(lista_chunks_vec_db)
    bm25 = BM25Okapi(tokenized_docs)

    # Traducir al inglés que es el idioma utilizado por la base de datos vectorial
    if detect(query_text) != 'en':
      query_text = GoogleTranslator(source=detect(query_text), target='en').translate(query_text)
    # Tokenizar la consulta
    tokenized_query = tokenize([query_text])[0]

    # Obtener las puntuaciones BM25 para la consulta
    scores = bm25.get_scores(tokenized_query)

    # Ordenar los resultados según las puntuaciones BM25
    sorted_indices = np.argsort(scores)[::-1]
    top_indices = sorted_indices[:top_n]

    return [(lista_chunks_vec_db[i], scores[i]) for i in top_indices]

In [102]:
def busqueda_semantica(query_text: str, top_n: int = 5)->list[tuple]:
    """
    Realiza una búsqueda semántica en la base de datos vectorial según la
    consulta recibida. Devuelve los trozos de documentos más relevantes
    según su similitud con la consulta.
    """
    # Consultar los embeddings de la consulta
    embed_consulta = embed([query_text]).numpy().tolist()

    # Realizar la consulta en la base de datos de vectores (chromadb)
    query_consulta = {
        'query_embeddings': embed_consulta,
        'n_results': top_n,  # Limitar los resultados a `top_n`
    }

    # Consultar en la base de datos
    results = vec_db.query(**query_consulta)

    # Obtener documentos y distancias
    documents = results.get('documents')
    distances = results.get('distances')

    # Combinar documentos y distancias en una lista de tuplas
    combined_results = [(documents[0][i], distances[0][i]) for i in range(len(documents[0]))]

    return combined_results

In [103]:
def busqueda_hibrida(query_text: str,top_n_palabra_clave:int=5,top_n_busqueda_semantica:int=5) -> list[tuple]:
    """
    Combina los resultados obtenidos por la búsqueda por palabras claves  y la
    búsqueda semántica. Devuelve los documentos más relevantes junto con sus
    respectivos puntajes.
    """
    # Obtener los resultados de BM25
    bm25_results = busqueda_palabras_claves(query_text,top_n_palabra_clave)

    # Obtener los resultados de la búsqueda semántica (ChromaDB)
    semantic_results = busqueda_semantica(query_text,top_n_busqueda_semantica)

    # Obtener los puntajes de BM25 para normalizarlos al rango 0-2
    bm25_scores = [bm25_score for _, bm25_score in bm25_results]
    min_bm25_score = min(bm25_scores)
    max_bm25_score = max(bm25_scores)

    # Crear un diccionario para combinar los puntajes
    combined_results = {}

    # Añadir los resultados de BM25 al diccionario, escalados al rango 0-2
    for doc, bm25_score in bm25_results:
        # Normalización Min-Max al rango 0-2
        if max_bm25_score != min_bm25_score:
            scaled_bm25_score = 2 * (bm25_score - min_bm25_score) / (max_bm25_score - min_bm25_score)
        else:
            scaled_bm25_score = 1  # Si todos los puntajes de BM25 son iguales, asignar un valor arbitrario (1 en este caso)

        combined_results[doc] = {'bm25': scaled_bm25_score, 'semantic': 0}

    # Añadir los resultados de la búsqueda semántica al diccionario
    for doc, semantic_distance in semantic_results:
        semantic_score = 2 - (semantic_distance)  # Convertir distancia a similitud (0 -> 2, 2 -> 0)
        if doc not in combined_results:
            combined_results[doc] = {'bm25': 0, 'semantic': semantic_score}
        else:
            combined_results[doc]['semantic'] = semantic_score

    # Crear una lista con los documentos y sus puntajes combinados
    busqueda_hibrida_docs = [
        (doc, combined_results[doc]['bm25'] + combined_results[doc]['semantic'])
        for doc in combined_results
    ]

    # Ordenar los resultados por el puntaje combinado
    busqueda_hibrida_docs_sorted = sorted(busqueda_hibrida_docs, key=lambda x: x[1], reverse=True)

    # Devolver los documentos re-rankeados
    return busqueda_hibrida_docs_sorted

In [104]:
def rerank_resultados(query: str, resultados:list[tuple], top_k: int = 5) -> list[tuple]:
  """
  Re-ranking usando cross-encoder
  """
  # Preparar pares para re-ranking
  pairs = [(query, doc) for doc,score in resultados]
  cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')
  # Calcular scores de re-ranking
  rerank_scores = cross_encoder.predict(pairs)
  idx_ordenados_asc = np.argsort(rerank_scores)[::-1]
  results = []
  for idx_top in idx_ordenados_asc[:top_k]:
    result = resultados[idx_top][0],rerank_scores[idx_top]
    results.append(result)
  return results

  # reranked_results = sorted(initial_results, key=lambda x: x.rerank_score, reverse=True)

  # Actualizar ranking final
  for i, result in enumerate(reranked_results):
      result.final_rank = i + 1

  return reranked_results

In [105]:
busqueda_semantica('¿Cuál es la mejor táctica para ganar la partida?')


[('Given the relatively short duration of a typical battle scenario, we recommend match play, with each player taking first one side, then the other in an immediate re-match.This helps balance any historical advantage that one side may have in a particular scenario. The winner of the match is theplayer who scores the most Victory Medals after both battles. 1 6+ 34Place Terrain hex. x20 x4 x912 10 8',
  1.1908817291259766),
 ("did after myfirst play I was already seeing strategyin the game but I will say it's sort ofit's sort of a game where by the end ofyour opponent's next turn any strategyyou may have had going into the game umwell it may have already gone out thewindow by the end of his turn which isone of my my favorite things about thisgame believe it or not it seems toalways keep you on your toes and reallyfocus",
  1.2089853286743164),
 ('as well as try to intro duce some of the tactic cards we already talked about. During the explanations we will not generally discuss good stra

In [106]:
busqueda_palabras_claves('¿Cuál es la mejor táctica para ganar la partida?')


[('SIDE66Object of the Game The object of the game is to be the first to win a set number of Victory Medals (usually 4 to 6, depending on the selected battle scenario’s victory conditions). In Memoir ’44, a Victory Medal is gained for each enemy unit entirely eliminated from the battlefield. Each Medal won is designated by placing the last figure of the eliminated unit on any of the available Medal',
  15.522866093666078),
 ("engaging in Close Assault with the enemy, in order to stop them from firing on another weaker unit that could earn them a medal. It also has ramifications for selecting a retreat path when flags are rolled. Know What You are Playing Memoir is a wonderful game in my opinion if you take it for what it is. Memoir '44 is not a 'War Game' in the truest sense of the term. Memoir is a Card Driven Game",
  15.257266742392204),
 ("Memoir is a Card Driven Game (CDG) and it is all about 'Hand Management'. The skill to Memoir is identifying the strength of the cards at your d

In [107]:
resultados_iniciales = busqueda_hibrida('¿Cuál es la mejor táctica para ganar la partida?')
resultados_iniciales


[('SIDE66Object of the Game The object of the game is to be the first to win a set number of Victory Medals (usually 4 to 6, depending on the selected battle scenario’s victory conditions). In Memoir ’44, a Victory Medal is gained for each enemy unit entirely eliminated from the battlefield. Each Medal won is designated by placing the last figure of the eliminated unit on any of the available Medal',
  2.0),
 ("engaging in Close Assault with the enemy, in order to stop them from firing on another weaker unit that could earn them a medal. It also has ramifications for selecting a retreat path when flags are rolled. Know What You are Playing Memoir is a wonderful game in my opinion if you take it for what it is. Memoir '44 is not a 'War Game' in the truest sense of the term. Memoir is a Card Driven Game",
  1.4007289100159295),
 ("Memoir is a Card Driven Game (CDG) and it is all about 'Hand Management'. The skill to Memoir is identifying the strength of the cards at your disposal and how

In [108]:
rerank_resultados('¿Cuál es la mejor táctica para ganar la partida?',resultados_iniciales)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning:


The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.



config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

[('as well as try to intro duce some of the tactic cards we already talked about. During the explanations we will not generally discuss good strategy , it’s up to the player to play, learn and discover this. Many of the moves will seem pointless, they will be chosen because I think t hey are important to show you HOW they should be played so, again, do not rely on correctness of gameplay strategy-',
  -9.09283),
 ('◆Remove all Tactic cards from the deck – just keep the Section cards ◆If needed, you can choose not to play the Take Ground and the Armor Overrun rules ◆Every time the deck is exhausted, remember to shuffle the Section cards before making a new deck It is easier if an adult introduces the children to the game, and plays a couple of games with them. It is also a good opportun ity for a parent or a',
  -9.304172),
 ('SIDE66Object of the Game The object of the game is to be the first to win a set number of Victory Medals (usually 4 to 6, depending on the selected battle scenari

### Query para base de datos de grafos

In [109]:
def get_query(input_text: str) -> str:
    """
    Genera una consulta SPARQL utilizando Qwen a partir de la entrada del usuario.
    """
    # Tu clave API de Hugging Face
    api_key = config('HUGGINGFACE_TOKEN', userdata.get('HUGGINGFACE_TOKEN'))
    client_HF = InferenceClient(api_key= api_key)
    # URL de la API de Hugging Face para la clasificación de texto
    api_url = "https://api-inference.huggingface.co/models/Qwen/Qwen2.5-72B-Instruct"

    chat_prompt = [{
        "role": "system",
        "content": f"""
        Eres un modelo que realiza consultas a una base de datos RDF sobre el juego de mesa "Memoir '44" a raíz de una frase del usuario.
        La base tiene la siguiente estructura: las entidades son {entidades}, las relaciones son {relaciones}.
        Asegúrate de diferenciar bien entre diseñadores y creadores.
        Debes escribir SOLAMENTE UNA consulta URI con SPARQL que se adecue a la frase del usuario.
        Es importante que solo respondas con una sola consulta.
        Por ejemplo, si recibes la frase: '¿Cuándo se creó el juego?' deberías solamente
        devolver: "SELECT (STRAFTER(STR(?Artist), 'http://memoir_44.org/') AS ?name) WHERE {{?game a <http://memoir_44.org/Game> .?game <http://memoir_44.org/tiene_como_artista> ?Artist .}}"
        Es importante que respetes todos los corchetes, llaves y entidades luego de ?."""
    }, {
        "role": "user",
        "content": input_text
    }]

    # Realizar la solicitud de completado de chat usando el cliente
    completion = client_HF.chat.completions.create(
        model="Qwen/Qwen2.5-72B-Instruct",
        messages=chat_prompt,
        max_tokens=500,
        temperature=0.1
    )

    # Extraer y devolver la consulta SPARQL generada
    return completion.choices[0].message['content']

In [110]:
# Obtener la consulta SPARQL generada
query = "¿Cuáles otros juegos diseñó el diseñador de Memoir?"
sparql_query = get_query(query)

# Imprimir consulta texto, consulta SPARQL, respuesta base de datos
print("consulta\n",query)
print("\nconsulta en lenguaje SPARQL\n",sparql_query)
print("\nrespuesta")
results = g.query(sparql_query)
for resultado in results:
  "En el caso que devuelva un objeto URIRef hay que separar por /"
  if isinstance(resultado[0],str):
    resultado = resultado[0].split('/')[-1]
    print(resultado)
  else:
    resultado.name

consulta
 ¿Cuáles otros juegos diseñó el diseñador de Memoir?

consulta en lenguaje SPARQL
 SELECT (STRAFTER(STR(?otherGame), 'http://memoir_44.org/') AS ?name) WHERE { <http://memoir_44.org/Memoir_44> <http://memoir_44.org/diseñado_por> ?designer . ?designer <http://memoir_44.org/diseñó> ?otherGame . FILTER(?otherGame != <http://memoir_44.org/Memoir_44>) }

respuesta
Memoir_%2744
Commands_%26_Colors%3A_Ancients
BattleLore%3A_Second_Edition
Commands_%26_Colors%3A_Napoleonics
BattleLore
Perudo


In [111]:
# Obtener la consulta SPARQL generada
query = "¿En cuál año salió Memoir?"
sparql_query = get_query(query)

# Imprimir consulta texto, consulta SPARQL, respuesta base de datos
print("consulta\n",query)
print("\nconsulta en lenguaje SPARQL\n",sparql_query)
print("\nrespuesta")
results = g.query(sparql_query)
for resultado in results:
  "En el caso que devuelva un objeto URIRef hay que separar por /"
  if isinstance(resultado[0],str):
    resultado = resultado[0].split('/')[-1]
    print(resultado)
  else:
    resultado.name

consulta
 ¿En cuál año salió Memoir?

consulta en lenguaje SPARQL
 SELECT ?year WHERE { <http://memoir_44.org/Memoir_44> <http://memoir_44.org/lanzado_en> ?year . }

respuesta
2004


### Query para base de datos tabular

In [112]:
def get_tabular_query(input_text: str) -> str:
    """
    Genera una consulta a un data frame de Pandas utilizando Qwen a partir de
    la entrada del usuario.
    """
    api_key = config('HUGGINGFACE_TOKEN', userdata.get('HUGGINGFACE_TOKEN'))
    client_HF = InferenceClient(api_key= api_key)
    # URL de la API de Hugging Face
    api_url = "https://api-inference.huggingface.co/models/Qwen/Qwen2.5-72B-Instruct"

    chat_prompt = [{
        "role": "system",
        "content": f"""
        Eres un modelo que realiza consultas a una base de datos tabular en dataframe de pandas sobre el juego de mesa "Rajas of the Ganjes" a raíz de una frase del usuario.
        La base tiene la siguiente estructura: las columnas son: {df_estadisticas.columns} y las entidades
        : {df_estadisticas['Entidad']}.
        Avg.rating contiene el puntaje medio del juego.
        Debes escribir SOLAMENTE UNA consulta de búsqueda en dataframe que se adecue a la frase del usuario.
        Es importante que solo respondas con una sola consulta.
        Por ejemplo, si recibes la frase: '¿Cuántos jugadores han jugado el juego?' deberías solamente
        devolver: 'df_estadisticas[df_estadisticas['Entidad'] == 'All Time Plays']['Valor'].values[0]'"""
    }, {
        "role": "user",
        "content": input_text
    }]

    # Realizar la solicitud de completado de chat usando el cliente
    completion = client_HF.chat.completions.create(
        model="Qwen/Qwen2.5-72B-Instruct",
        messages=chat_prompt,
        max_tokens=500,
        temperature=0.1
    )
    return completion.choices[0].message['content']

In [113]:
eval(get_tabular_query('¿cuántas comentarios tiene el juego?'))


'7,277'

In [114]:
query = get_tabular_query('¿Cuántos jugadores han jugado el juego?')

In [115]:
eval(query)

'172,716'

In [116]:
get_tabular_query('¿Cuántos jugadores han jugado el juego?')

"df_estadisticas[df_estadisticas['Entidad'] == 'All Time Plays']['Valor'].values[0]"

## Parte 6: Implementación RAG

In [117]:
class llm_memoir_44():
    """
    Esta clase define un chatbot interactivo diseñado para responder consultas sobre el juego *Memoir '44*
    utilizando un modelo de lenguaje y diferentes fuentes de datos.

    La clase permite realizar consultas, obtener contextos de diversas bases de datos (como bases de datos vectoriales,
    gráficas o tabulares), y generar un prompt para interactuar con un modelo de lenguaje.

    Métodos:
    - __init__: Inicializa el cliente de inferencia y define los atributos necesarios para el chatbot.
    - set_context: Obtiene el contexto necesario para responder a una consulta, basándose en el tipo de base de datos.
    - make_prompt: Crea un prompt estructurado para la interacción con el modelo de lenguaje.
    - hablar: Inicia una conversación con el chatbot, permite al usuario hacer consultas y recibir respuestas del modelo.
    """

    def __init__(self):
        """
        Inicializa el cliente de inferencia (InferenceClient) con una clave de API y establece los atributos
        iniciales para el chatbot.
        """
        api_key = config('HUGGINGFACE_TOKEN', userdata.get('HUGGINGFACE_TOKEN'))
        self.client_hugging_face = InferenceClient(api_key=api_key)
        self.info = None
        self.chat_prompt = None
        self.historial = []

    def set_context(self, consulta):
        """
        Obtiene el contexto adecuado para una consulta dada, determinando el tipo de base de datos (vectorial, gráfica o tabular).

        Dependiendo de la base de datos asignada, la función consulta y recupera los datos necesarios
        para proporcionar una respuesta relevante.

        Parámetros:
        - consulta (str): La consulta del usuario para la que se necesita obtener el contexto.

        Setea el atributo info de la clae
        """
        db = clasificador_llm_predict(consulta)
        if db == 'vector_db':
            resultados_iniciales = busqueda_hibrida(consulta)
            context = rerank_resultados(consulta,resultados_iniciales)
        elif db == 'graph_db':
            query = get_query(consulta)
            resultados = g.query(query)
            context = []
            for resultado in resultados:
              if isinstance(resultado[0],str):
                resultado = resultado[0].split('/')[-1]
                context.append(resultado)
              else:
                context.append(resultado)
        elif db == 'tabular_db':
            query = get_tabular_query(consulta)
            context = [eval(query)]
        else:
            context = None

        self.info = context

    def make_prompt(self, consulta):
        """
        Crea un prompt estructurado para interactuar con el modelo de lenguaje, utilizando el contexto
        obtenido de la base de datos y la consulta del usuario.

        Parámetros:
        - consulta (str): La consulta del usuario para la que se necesita generar el prompt.

        Setea el atributo chat_prompt que pasará a ser una lista de diccionarios la cual contiene
        el prompt estructurado con el rol de sistema y usuario para la interacción con el modelo.
        """
        prompt = []
        primer_elemento_prompt = {
          "role": "system",
          "content": f"""
              Eres un modelo que responde preguntas acerca del juego *Memoir '44*.
              Es importante que sólo respondas en base a la siguiente información: {self.info}.
              Es muy importante que contestes en oración. Si la información proporcionada es un número, responde con ese número como parte de la oración.
              Sólo si la información es "None", responde 'No tengo información para responder la consulta. Puede consultarme sobre otro tema'."""
        }
        prompt.append(primer_elemento_prompt)
        if self.historial != []:
          for pregunta,respuesta in self.historial:
            diccionario_user = {"role": "user", "content": pregunta}
            diccionario_assistant = {"role": "assistant", "content": respuesta}
            prompt.append(diccionario_user)
            prompt.append(diccionario_assistant)
        ultimo_elemento_prompt ={
            "role": "user",
            "content": consulta
        }
        prompt.append(ultimo_elemento_prompt)
        self.chat_prompt = prompt

    def hablar(self):
        """
        Inicia una conversación interactiva con el usuario, permitiendo realizar consultas y recibir respuestas
        generadas por el modelo de lenguaje. El ciclo de conversación se mantiene hasta que el usuario escribe
        'chau' para salir.

        El ciclo de interacción involucra la obtención de contexto, la creación del prompt y la generación de respuestas
        del modelo, que se imprimen como respuestas del agente.
        """
        consulta = 'c'
        print('Bienvenido a Memoir 44 Chatbot! para salir escribe "chau"')
        while True:
            consulta = input("Escribe tu consulta, para salir despídete con chau:")
            if consulta == "Chau" or consulta == "chau" or consulta == " chau":
              print('Gracias por usar Memoir 44 Chatbot, hasta la próxima!')
              break
            print(f"User: {consulta}")
            self.set_context(consulta)
            self.make_prompt(consulta)
            # Realizar la solicitud de completado de chat usando el cliente
            completion = self.client_hugging_face.chat.completions.create(
                model="Qwen/Qwen2.5-72B-Instruct",
                messages=self.chat_prompt,
                max_tokens=400, # Se limita la extensión de la respuesta para evitar textos excesivamente largos
                temperature=0.4 # Se permite cierto grado de alucinación tolerable
            )
            resp = completion.choices[0].message['content']
            intercambio = (consulta,resp)
            self.historial.append(intercambio)
            if detect(resp) != 'es':
              resp = GoogleTranslator(source=detect(resp), target='es').translate(resp)
            print(f"Agente: {resp}")

In [118]:
llm = llm_memoir_44()


In [119]:
#llm.hablar()

# Ejercicio 2 - Agente

In [120]:
# Descarga de Ollama
!curl -fsSL https://ollama.com/install.sh | sh

# Iniciamos Ollama en background
!rm -f ollama_start.sh
!echo '#!/bin/bash' > ollama_start.sh
!echo 'ollama serve' >> ollama_start.sh
# Make the script executable
!chmod +x ollama_start.sh
!nohup ./ollama_start.sh &

!ollama pull llama3.2 > ollama.log

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
nohup: appending output to 'nohup.out'
pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠼ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠧ pulling manifest ⠧ pulling manifest 
pulling dde5aa3fc5ff...   0% ▕▏    0 B/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   0% ▕▏    0 B/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   0% ▕▏ 5.9 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   2% ▕▏  33 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   4% ▕

In [121]:
!ollama list


NAME               ID              SIZE      MODIFIED               
llama3.2:latest    a80c4f17acd5    2.0 GB    Less than a second ago    


In [122]:
import logging

# Configuración del logger
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

In [123]:
def get_info_graph_db(consulta):
  query = get_query(consulta)
  resultados = g.query(query)
  info = []
  for resultado in resultados:
    if isinstance(resultado[0],str):
      resultado = resultado[0].split('/')[-1]
      info.append(resultado)
    else:
      info.append(resultado)
  query = get_query(consulta)
  info = [result.name[0:] for result in g.query(query)]
  return info


def get_info_tabular_db(consulta):
  query = get_tabular_query(consulta)

  info = [eval(query), query]
  return info


def get_info_vector_db(consulta):
  resultados_iniciales = busqueda_hibrida(consulta)
  info = rerank_resultados(consulta,resultados_iniciales)
  return info


In [124]:
# Crear las herramientas para el agente
tools_list = [
    FunctionTool.from_defaults(fn=get_info_graph_db, description="Busca información en la base de datos de grafos. Usar: texto de consulta"),
    FunctionTool.from_defaults(fn=get_info_tabular_db, description="Busca información en la base de datos tabular. Usar: texto de consulta"),
    FunctionTool.from_defaults(fn=get_info_vector_db, description="Busca información en la base de datos vectorial. Usar: texto de consulta"),
]

In [125]:
# Configurar el LLM de Ollama para usar Llama 3.2
llm = Ollama(
    model="llama3.2:latest",
    request_timeout=100.0,  # Aumentado el timeout
    temperature=0.1,  # Reducida la temperatura para respuestas más deterministas
    context_window=4096  # Aumentado el contexto
)


Settings.llm = llm
import time
!nohup litellm --model ollama/llama3.2:latest --port 8000 > litellm.log 2>&1 &

In [126]:
agent = ReActAgent.from_tools(
    tools_list,
    llm=llm,
    verbose=True,
    chat_formatter=ReActChatFormatter(),
    system_prompt="""Estás diseñado para responder preguntas acerca del juego 'Memoir 44' utilizando **exclusivamente información de las bases de datos tabular, vectorial y de grafos, brindada por las herramientas que se mencionana a continuación***. Asegúrate de seguir estrictamente las instrucciones para cada consulta.

          ## Herramientas disponibles:
          1. **get_info_graph_db**: Busca información en la base de datos de grafos. Parámetro: texto de consulta.
          2. **get_info_tabular_db**: Busca información en la base de datos tabular. Parámetro: texto de consulta.
          3. **get_info_vector_db**: Busca información en la base de datos vectorial. Parámetro: texto de consulta.

          ### Temáticas de cada base:
          - **Grafos**: DISEÑADORES, CREADORES, PUBLICADORES DEL JUEGO, AÑO DE CREACIÓN DEL JUEGO, MECANISMOS.
          - **Tabular**: ESTADÍSTICA GENERAL.
          - **Vectorial**: REGLAS GENERALES, ESTRATEGIAS, OPINIONES, TACTICAS .

          ### Instrucciones para cada consulta:
          1. **Analiza la consulta** para determinar la información necesaria.
          2. **Usa las herramientas disponibles una por una**, pasando como parámetro **exactamente** la consulta recibida.
          3. **Combina los resultados** en una respuesta final clara y concisa.

          ### Reglas adicionales:
          - **Nunca modifiques la consulta del usuario**: Cada vez que recibas una consulta nueva, **olvida la anterior**. La nueva consulta debe ser tratada de manera independiente y sin influencias previas.
          - **Prioriza las herramientas** según la relación temática con la consulta. Asegúrate de seleccionar la herramienta que mejor se ajuste a la consulta de acuerdo con las temáticas proporcionadas.
          - Si la pregunta requiere información de **más de una herramienta**, usa **todas** las herramientas necesarias para obtener una respuesta completa.
          - **Nunca** respondas con información que no provenga directamente de las herramientas.
          - El formato de salida debe ser **estricto** y seguir el ejemplo que se proporciona.

          ### Ejemplo de interacción:
          #### Consulta 1:
          "¿Qué opinión hay del juego?"
          - **Pensamiento (Thought)**: Necesito buscar información sobre las reglas del juego.
          - **Acción (Action)**: `get_info_vector_db`
          - **Entrada de acción (Action Input)**: "¿Cuándo finaliza la partida?"
          - **Observación (Observation)**: "Memoir ’44 es el juego perfecto de iniciación a los wargames tácticos, con un sistema tremendamente asequible, divertido y ágil que lo hace apto para todos los públicos sin mermar demasiado el componente de simulación histórica..."
          - **Respuesta final (Final Answer)**: Memoir ’44 es el juego perfecto de iniciación a los wargames tácticos, con un sistema tremendamente asequible, divertido y ágil que lo hace apto para todos los públicos sin mermar demasiado el componente de simulación histórica.

          #### Consulta 2:
          "Año de creación del juego?"
          - **Pensamiento (Thought)**: Necesito buscar información sobre el año de creación del juego.
          - **Acción (Action)**: `get_info_graph_db`
          - **Entrada de acción (Action Input)**: "Año de creación del juego"
          - **Observación (Observation)**: "El juego fue creado en 2004."
          - **Respuesta final (Final Answer)**: El juego fue creado en 2004.

          ### IMPORTANTE:
          - **No uses información de consultas anteriores** para la nueva consulta.
          - **Nunca alteres la consulta recibida**. Siempre debes utilizar el texto exacto como parámetro.
          - **Siempre asegúrate de usar todas las herramientas necesarias** antes de no dar una respuesta completa.
          """
          ,
                react_chat_history=False,
              context="""
              Eres un asistente útil que responde en español. Recuerda las salidas de todas las herramientas para usarlas en la respuesta final.
              """)


# Función para interactuar con el agente
def chat_con_agente():
    """
    Función para interactuar con el agente ReAct.
    """
    query = input("")
    while query != 'chau':
      try:
          if not query.strip():
              return "La consulta está vacía"

          response = agent.chat(query)
          print(response)
      except Exception as e:
          print(f"Error al procesar la consulta: {str(e)}")
      query = input("")

* "¿Qué táctica se puede implementar para ganar el juego",
* "Año de creación del juego?",
* "¿Quién fue el creador del juego?",
* "¿son muchas las personas que quieren jugarlo?",
* "Qué puntaje obtuvo el juego?",


In [128]:
chat_con_agente()

¿Qué táctica se puede implementar para ganar el juego
> Running step b33218b7-36e9-4c09-9a7a-6dcdfb2caf70. Step input: ¿Qué táctica se puede implementar para ganar el juego
Thought: Puedo utilizar la herramienta get_info_graph_db para buscar información sobre las tácticas en el juego Memoir '44.
Action: get_info_graph_db
Action Input: {'consulta': "tácticas para ganar juego Memoir '44"}
Observation: Error: Expected {SelectQuery | ConstructQuery | DescribeQuery | AskQuery}, found '`'  (at char 0), (line:1, col:1)
> Running step 947aaf87-dcf3-4358-b871-ab7325bf5c82. Step input: None
Thought: Parece que la herramienta get_info_graph_db no aceptó la consulta. Necesito intentar con una consulta diferente.
Action: get_info_tabular_db
Action Input: {'consulta': "¿Quién es el creador del juego Memoir '44?"}
Observation: Error: invalid syntax (<string>, line 1)
> Running step efad32e5-cb69-4220-a28e-63d3318f8472. Step input: None
Thought: La herramienta get_info_tabular_db tampoco aceptó la con